#Deep learning solution for classification of documents


##1.1 Explanation
We would be using various deeplearning algorithms to try and classify our classes between letters, memos and forms

As usual, we have to get our extracted data fro google drive 

In [1]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '.zip'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title extractedData.zip, id 1lJYTmX_PUyUVDSWvAiFkmoEJWZNA3msH
title data.zip, id 1pMEgJWXyKopKo5OQTSpG1pTK8XFj7Cg9
title firebase-arduino-master.zip, id 0B4tww7CJlZPyUTd1enFxNklKOE0
title Re Zero Kara Hajimeru v09.zip, id 0B-1T9fhvtxApbjRIM2NZcGdjVFE
title Re Zero Kara Hajimeru v08.zip, id 0B-1T9fhvtxApRWZhaXpkSEZ4S1E


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1lJYTmX_PUyUVDSWvAiFkmoEJWZNA3msH'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('extractedData.zip')

In [0]:
!unzip extractedData.zip

In [5]:
import os

path = os.getcwd()  
print ("The current working directory is {}" .format(path))

The current working directory is /content


In [0]:
classes = ["Email","Letter","Memo"]; 
dataFolders = ["train","test","val"];

In [0]:
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [0]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

In [22]:
# For reproducibility
np.random.seed(1237)
 
# train file directory
path_train = os.path.join(path, "extractedData", "train")
 
files_train = skds.load_files(path_train,load_content=False)
 
train_label_index = files_train.target
train_label_names = files_train.target_names
train_labelled_files = files_train.filenames

 
train_labelled_text = []

# Read and add data from file to a list
i=0

for file in train_labelled_files:
  with open(file, 'r') as f:
    text = f.read() 
    train_labelled_text.append(text)
  i += 1
    
print(len(train_label_index))
print(train_label_index[:10])

print(train_label_names)
print(train_labelled_text[:10])


1249
[1 2 0 2 0 2 0 0 2 2]
['Email', 'Letter', 'Memo']
["@\n\n \n\n\xc2\xbb.DEC-04-97 THU 11:17 AM NB BULK PACKAGING FAX NO. 7704293533 P, 05/08\n\nUnited States Food Safety Washington, 0.C.\nDepartment of and Inspection 20250\nAgriculture Service\n\nEgg Products Inspection Division\nJune 14, 1996\n\nMr. Gary Brown\nMacMillan Bloedel Bulk Packaging\nDivision of MacMillan Bloedel of America, Inc.\n\n1811 West Oak Parkway\nMarietta, Georgia 30062\n\nDear Mr, Brown:\n\nThis is in response to your June 4 letter requesting Department of Agriculmre (USDA)\napproval of the polypropylene Tre Blue Ball Valve, manufactured by BLDL, Inc, Totowa,\nNew Jersey, for use in plants operating under the federal egg products inspection program.\nYou propose to use the apparatus as an outlet valve for attachment to your firm's single-use\ncorrugated fiber IBC portable container approved for use in official egg products plants,\nEnclosed with your letter were specifications and drawings describing the const